# Data Transformation

The following code aims to perform several data transformation process in order to clean and improve the normalization of the data available in the movies_dataset.csv and credits.csv files.

The process is divided into stages and steps for a simplier understanding of the main objective of the actions performed.

## Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import json

## Load the Dataset Files

In [ ]:
# Load the data from the Datasets folder

filepath = '/content/drive/MyDrive/Notebooks/Henry/PI/MLOps/datasets/movies_dataset.csv'

df_movies = pd.read_csv(filepath)

filepath = '/content/drive/MyDrive/Notebooks/Henry/PI/MLOps/datasets/credits.csv'

df_credits = pd.read_csv(filepath)

<ipython-input-3-47f636081aea>:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv(filepath)


## Analyzing Issues in the Movies DF

In [ ]:
# First lets take a loot at the columns of our dataset
df_movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


There are some complex data structures in some of the columns. Let's analyze their meaning:

Feature Description
* adult: Indicates if the movie is X-rated, adult-only.
* belongs_to_collection: A **dictionary** that indicates to which movie franchise or series the movie belongs.
* budget: The budget of the film, in dollars
* genres: A dictionary indicating all the genres associated with the movie.
* homepage: The official website of the movie
* id: ID of the movie
* imdb_id : IMDB ID of the film
* original_language: Original language in which the movie was recorded
* original_title: Original title of the movie
* overview: Short summary of the movie
* popularity: Popularity Score of the movie, assigned by TMDB (TheMoviesDataBase)
* poster_path: URL of the movie poster
* production_companies: **List** of production companies associated with the movie
* production_countries: **List** of countries where the movie was produced
* release_date Release date of the movie
* revenue : Gross revenue of the film, in dollars
* runtime : Duration of the film, in minutes
* spoken_languages: **List** of languages spoken in the movie
* status : Status of the current movie (if it was announced, if it has been released, etc.)
* tagline : Phrase associated with the movie
* title : Title of the movie
* video : Indicates whether or not there is a video trailer available on TMDB
* vote_average : Average score of reviews of the film
* vote_count : Number of votes received by the movie, in TMDB

### 1. Dictionaries

Let's start working on those features that are stored as Dictionaries. The aim is to achieve a more normalized stage of our data.

#### 1.1 Belongs to Collection

It's featured stored as a dictionary with 4 keys:

* id
* name of the collection
* poster path
* backdrop_path

In my opinion, an excellent way to normalize this data is to create a new table or df associated to the collections. And only store the id of the collections in the rows of our df_movies.

The problem here is that  if some of the keys in the dictionaries have None values, that could cause an error when using json.loads() to convert the string representation to a dictionary. This is because None is not a valid value in JSON.

One way to handle this is to replace the None values in the string representation with a string that can be converted to None after using json.loads(). For example, you could replace None with the string "null", which is a valid value in JSON that can be converted back to None in Python.

In [ ]:
filepath = '/content/drive/MyDrive/Notebooks/Henry/PI/MLOps/datasets/movies_dataset.csv'

df_movies = pd.read_csv(filepath)

# Replace undesired strings
df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].str.replace("':", '\":')
df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].str.replace(" '", '\"')
df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].str.replace("{'", '{"')
df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].str.replace("',", '",')
df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].str.replace("'}", '"}')
df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].str.replace('e"E', "e'E")
df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].str.replace('s"n', "s'n")

# Replace None values with the string "null" in the 'collections' column
df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].str.replace("None", "null")

# Convert the strings representing dictionaries to actual dictionaries
df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].apply(lambda x: json.loads(x) if pd.notna(x) else np.nan)

<ipython-input-62-8826836cce82>:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv(filepath)
<ipython-input-62-8826836cce82>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].str.replace("{'", '{"')
<ipython-input-62-8826836cce82>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].str.replace("'}", '"}')


In [ ]:
isinstance(df_movies['belongs_to_collection'].iloc[0], dict)

True

In [ ]:
# Filter out rows where the 'collections' column is not a dictionary
df_not_dict = df_movies[~df_movies['belongs_to_collection'].apply(lambda x: isinstance(x, dict))]
df_not_dict.dropna()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count


In [ ]:
# Get all unique dictionaries in the 'collections' column, dropping NaN values
unique_collections = df_movies['belongs_to_collection'].dropna()
unique_collections = unique_collections.dropna()
df_collections = unique_collections.apply(pd.Series)

In [ ]:
df_collections

,id,name,poster_path,backdrop_path,0
0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,NaN
2,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg,NaN
4,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg,NaN
9,645.0,James Bond Collection,/HORpg5CSkmeQlAolx3bKMrKgfi.jpg,/6VcVl48kNKvdXOZfJPdarlUGOsk.jpg,NaN
12,117693.0,Balto Collection,/w0ZgH6Lgxt2bQYnf1ss74UvYftm.jpg,/9VM5LiJV0bGb1st1KyHA3cVnO2G.jpg,NaN
...,...,...,...,...,...
45355,37261.0,The Carry On Collection,/2P0HNrYgKDvirV8RCdT1rBSJdbJ.jpg,/38tF1LJN7ULeZAuAfP7beaPMfcl.jpg,NaN
45358,37261.0,The Carry On Collection,/2P0HNrYgKDvirV8RCdT1rBSJdbJ.jpg,/38tF1LJN7ULeZAuAfP7beaPMfcl.jpg,NaN
45369,37261.0,The Carry On Collection,/2P0HNrYgKDvirV8RCdT1rBSJdbJ.jpg,/38tF1LJN7ULeZAuAfP7beaPMfcl.jpg,NaN
45371,477208.0,DC Super Hero Girls Collection,None,None,NaN


In [ ]:
df_collections.drop(0, axis=1, inplace=True)
df_collections.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4494 entries, 0 to 45382
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             4491 non-null   float64
 1   name           4491 non-null   object 
 2   poster_path    3948 non-null   object 
 3   backdrop_path  3263 non-null   object 
dtypes: float64(1), object(3)
memory usage: 175.5+ KB


In [ ]:
# Drop rows where 'id' is NaN
df_collections = df_collections.dropna(subset=['id'])

In [ ]:
df_collections.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4491 entries, 0 to 45382
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             4491 non-null   float64
 1   name           4491 non-null   object 
 2   poster_path    3948 non-null   object 
 3   backdrop_path  3263 non-null   object 
dtypes: float64(1), object(3)
memory usage: 175.4+ KB


In [ ]:
df_collections['id'] = df_collections['id'].astype(int)
df_collections.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4491 entries, 0 to 45382
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             4491 non-null   int64 
 1   name           4491 non-null   object
 2   poster_path    3948 non-null   object
 3   backdrop_path  3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 175.4+ KB


<ipython-input-154-203b5dba4e60>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_collections['id'] = df_collections['id'].astype(int)


In [ ]:
df_collections.head()

,id,name,poster_path,backdrop_path
0,10194,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
2,119050,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
4,96871,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg
9,645,James Bond Collection,/HORpg5CSkmeQlAolx3bKMrKgfi.jpg,/6VcVl48kNKvdXOZfJPdarlUGOsk.jpg
12,117693,Balto Collection,/w0ZgH6Lgxt2bQYnf1ss74UvYftm.jpg,/9VM5LiJV0bGb1st1KyHA3cVnO2G.jpg


Now, We only need to replace the column in df_movies for only the id of the collection.

In [ ]:

# Cast the 'collections' column as string
df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].astype(str)
# Trim the 'id' value from the 'collections' column
df_movies['collection_id'] = df_movies['belongs_to_collection'].apply(lambda x: x.split("'id': ")[1].split(',')[0] if pd.notna(x) and "'id': " in x else np.nan)


In [ ]:
df_movies = df_movies.drop('belongs_to_collection', axis=1)
df_movies.head()

,adult,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,collection_id
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,10194
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN
2,False,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,119050
3,False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,NaN
4,False,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,96871


#### 2. Fill null values in revenue and Budget

In [ ]:
# Fill the NaN values in the 'revenue' and 'budget' columns with 0
df_movies['revenue'] = df_movies['revenue'].fillna(0)
df_movies['budget'] = df_movies['budget'].fillna(0)

df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   adult                 45466 non-null  object 
 1   budget                45466 non-null  object 
 2   genres                45466 non-null  object 
 3   homepage              7782 non-null   object 
 4   id                    45466 non-null  object 
 5   imdb_id               45449 non-null  object 
 6   original_language     45455 non-null  object 
 7   original_title        45466 non-null  object 
 8   overview              44512 non-null  object 
 9   popularity            45461 non-null  object 
 10  poster_path           45080 non-null  object 
 11  production_companies  45463 non-null  object 
 12  production_countries  45463 non-null  object 
 13  release_date          45379 non-null  object 
 14  revenue               45466 non-null  float64
 15  runtime            

#### 3. Drop rows that do not have been released yet

In [ ]:
df_movies = df_movies.dropna(subset=['release_date'])

df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45379 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   adult                 45379 non-null  object 
 1   budget                45379 non-null  object 
 2   genres                45379 non-null  object 
 3   homepage              7769 non-null   object 
 4   id                    45379 non-null  object 
 5   imdb_id               45365 non-null  object 
 6   original_language     45368 non-null  object 
 7   original_title        45379 non-null  object 
 8   overview              44438 non-null  object 
 9   popularity            45377 non-null  object 
 10  poster_path           45040 non-null  object 
 11  production_companies  45379 non-null  object 
 12  production_countries  45379 non-null  object 
 13  release_date          45379 non-null  object 
 14  revenue               45379 non-null  float64
 15  runtime            

#### 6. Drop features that will not be used

In [ ]:
columns_to_drop = ['video',
                   'imdb_id',
                   'adult',
                   'original_title',
                   'poster_path',
                   'homepage']

df_movies = df_movies.drop(columns_to_drop, axis=1)

df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45379 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                45379 non-null  object 
 1   genres                45379 non-null  object 
 2   id                    45379 non-null  object 
 3   original_language     45368 non-null  object 
 4   overview              44438 non-null  object 
 5   popularity            45377 non-null  object 
 6   production_companies  45379 non-null  object 
 7   production_countries  45379 non-null  object 
 8   release_date          45379 non-null  object 
 9   revenue               45379 non-null  float64
 10  runtime               45130 non-null  float64
 11  spoken_languages      45376 non-null  object 
 12  status                45296 non-null  object 
 13  tagline               20398 non-null  object 
 14  title                 45376 non-null  object 
 15  vote_average       

#### 4. Set dates format

In [ ]:
# Convert the 'release_date' column to a datetime format

# Use a try-except block to catch the error when converting the 'release_date' column to a datetime format
try:
    df_movies['release_date'] = pd.to_datetime(df_movies['release_date'],
                                           yearfirst=True,
                                           format = '%Y-%m-%d')
except ValueError as e:
    print('ValueError:', e)
    print('Row with the error:', df_movies.loc[df_movies['release_date'] == '1'])




ValueError: time data "1" at position 10394 doesn't match format specified
Row with the error:                                  budget  \
19730  /ff9qCepilowshEtG2GYWwzt2bs4.jpg   

                                                  genres          id  \
19730  [{'name': 'Carousel Productions', 'id': 11176}...  1997-08-20   

      original_language  overview popularity production_companies  \
19730             104.0  Released        NaN                False   

      production_countries release_date  revenue  runtime spoken_languages  \
19730                  6.0            1      0.0      NaN              NaN   

      status tagline title  vote_average  vote_count collection_id  
19730    NaN     NaN   NaN           NaN         NaN           NaN  


There was a human error when entrying some rows, let's try fixing it

In [ ]:
df_movies.loc[df_movies['release_date'] == '1', ['id', 'release_date']] = ['1', '1997-08-20']

In [ ]:
df_movies.loc[df_movies['release_date'] == '12', ['id', 'release_date']] = ['12', '2012-09-29']

In [ ]:
df_movies.loc[df_movies['release_date'] == '22', ['id', 'release_date']] = ['22', '2014-01-01']

In [ ]:
try:
    df_movies['release_date'] = pd.to_datetime(df_movies['release_date'],
                                           yearfirst=True,
                                           format = '%Y-%m-%d')
except ValueError as e:
    print('ValueError:', e)



In [ ]:
# Create a new column 'release_year' by extracting the year value from the 'release_date' column
df_movies['release_year'] = df_movies['release_date'].dt.year

df_movies.head()

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,collection_id,release_year
0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,10194,1995
1,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,NaN,1995
2,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,119050,1995
3,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,NaN,1995
4,0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,96871,1995


#### 5. Return on investment column

In [ ]:
# Create a new column 'return' with the ROI by dividing 'revenue' by 'budget', and setting the 'return' value to 0 if 'budget' is 0
df_movies['budget'] = df_movies['budget'].astype(int)
df_movies['return'] = df_movies['revenue'] / df_movies['budget']
df_movies['return'] = df_movies['return'].apply(lambda x: 0 if pd.isna(x) or x == float('inf') or x == float('-inf') or x == float('nan') or x == 0 else x)

In [ ]:
try:
  df_movies['budget'].astype(int).unique()
except ValueError as e:
    print('ValueError:', e)
    print('Row with the error:', df_movies.loc[df_movies['budget'] == '/ff9qCepilowshEtG2GYWwzt2bs4.jpg'])

There are some uncorrected entried values, I choose to drop them as the title of the movie is unknown, so there is no way to solved those missing values.

In [ ]:
df_movies.drop(df_movies[df_movies['budget'] == '/ff9qCepilowshEtG2GYWwzt2bs4.jpg'].index, inplace = True)
df_movies.drop(df_movies[df_movies['budget'] == '/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg'].index, inplace = True)
df_movies.drop(df_movies[df_movies['budget'] == '/zaSf5OG7V8X8gqFvly88zDdRm46.jpg'].index, inplace = True)



#### 1.2 Genres

Similarly, the aims is to normalize the genre column by creating a df where the features of each genre are stored and only store the id of the genres associated to each movie in the df_movies

##### Creating the df_genres

In [ ]:
import json
df_movies['genres'] = df_movies['genres'].str.replace("'", '"')
# Convert the strings representing dictionaries to actual dictionaries
df_movies['genres'] = df_movies['genres'].apply(lambda x: json.loads(x) if pd.notna(x) else np.nan)

In [ ]:
df_movies['genres'].iloc[0]

[{'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'}]

In [ ]:
# Get all unique dictionaries in the 'collections' column, dropping NaN values
unique_genres = df_movies['genres'].dropna()
unique_genres = unique_genres.dropna()

df_genres = unique_genres.apply(pd.Series)

<ipython-input-12-e6f5e90f190d>:6: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_genres = unique_genres.apply(pd.Series)


In [ ]:
df_genresaux = df_genres[0]
for i in range(1,8):
  df_genresaux.append(df_genres[i])

df_genresaux = df_genresaux.dropna()
#Now that it is all concated in one single serie, lets turn it into a df with 2 features
df_genres = df_genresaux.apply(pd.Series)

<ipython-input-19-7642c5c994e8>:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_genresaux.append(df_genres[i])
<ipython-input-19-7642c5c994e8>:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_genresaux.append(df_genres[i])
<ipython-input-19-7642c5c994e8>:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_genresaux.append(df_genres[i])
<ipython-input-19-7642c5c994e8>:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_genresaux.append(df_genres[i])
<ipython-input-19-7642c5c994e8>:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_genres

In [ ]:
# Now drop duplicates
df_genres = df_genres.drop_duplicates()
df_genres.head()

,id,name
0,16,Animation
1,12,Adventure
2,10749,Romance
3,35,Comedy
5,28,Action


##### Normalize the genre column in df_movies

In [ ]:
#Now  we have to normalize the genres column
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45376 entries, 0 to 45375
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                45376 non-null  int64  
 1   genres                45376 non-null  object 
 2   id                    45376 non-null  int64  
 3   original_language     45365 non-null  object 
 4   overview              44435 non-null  object 
 5   popularity            45376 non-null  float64
 6   production_companies  45376 non-null  object 
 7   production_countries  45376 non-null  object 
 8   release_date          45376 non-null  object 
 9   revenue               45376 non-null  float64
 10  runtime               45130 non-null  float64
 11  spoken_languages      45376 non-null  object 
 12  status                45296 non-null  object 
 13  tagline               20398 non-null  object 
 14  title                 45376 non-null  object 
 15  vote_average       

In [ ]:
#Fortunately there are no nulls in the genres column
# Define a function to extract the 'id' value from each dictionary in the list
def extract_ids(genres):
    return [genre['id'] for genre in genres]

# Convert the strings representing dictionaries to actual dictionaries
df_movies['genres'] = df_movies['genres'].str.replace("'", '"')
df_movies['genres'] = df_movies['genres'].apply(lambda x: json.loads(x) if pd.notna(x) else np.nan)
# Apply the function to the 'genres' column to create a new column with a list of ids
column = 'genres'
df_movies[column+'_'+ 'ids'] = df_movies[column].apply(extract_ids)


,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,...,spoken_languages,status,tagline,title,vote_average,vote_count,collection_id,release_year,return,genre_ids
0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,10194.0,1995,12.451801,"[16, 35, 10751]"
1,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,NaN,1995,4.043035,"[12, 14, 10751]"
2,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.712900,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,119050.0,1995,0.000000,"[10749, 35]"
3,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,NaN,1995,5.090760,"[35, 18, 10749]"
4,0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,96871.0,1995,0.000000,[35]


#### 1.3 Production_Companies

A similar process is followed hereafter for Production_companies column.

In [ ]:
df_movies['production_companies'].iloc[453]

'[{"name": "Columbia Pictures", "id": 5}]'

In [ ]:
import unicodedata

# Define a function to remove non-ASCII characters from the string
def remove_non_ascii(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')

def parse_json(x):
    try:
        return json.loads(x)
    except json.JSONDecodeError:
        return np.nan

def normalize_column(df, column='column_to_normalize', key='id'):


    # Fix dictionaries format for Json library
    # Apply the function to the columns you want to map
    df[column] = df[column].apply(remove_non_ascii)
    df[column] = df[column].str.replace('"', "'")


    df[column] = df[column].str.replace("{'name': '", '{"name": "')
    df[column] = df[column].str.replace("', 'id':", '", "id":')
    df[column] = df[column].str.replace("', 'id':", '", "id":')
    df[column] = df[column].str.replace(", 'id':", ', "id":')
    df[column] = df[column].str.replace("{'name': ", '{"name": ')
    df[column] = df[column].str.replace("\xa0Check Entertainment", 'Check Entertainment')


    # Convert the strings representing dictionaries to actual dictionaries
    # Try to parse the JSON string and append the resulting dictionary to the list
    df[column] = df[column].apply(lambda x: (print(x),parse_json(x)) if pd.notna(x) else np.nan)
    # Get all unique dictionaries in the column, dropping NaN values
    aux = df[column].dropna()
    # Create a new dataframe with unique values of the column to normalize
    new_df = aux.apply(pd.Series)
    new_df_aux = new_df[0]
    for i in range(1,new_df.shape[1]):
      new_df_aux.append(new_df[i])

    new_df_aux = new_df_aux.dropna()
    #Now that it is all concated in one single serie, lets turn it into a df with 2 features
    new_df = new_df_aux.apply(pd.Series)
    new_df = new_df.drop_duplicates()
    # Create new column in the original dataframe with the keys of the column to normalize
    #df[column + '_' + key] = df[column].apply(lambda x: extract_ids(x,key=key) if x else [])
    return new_df



# Testing
#filepath = '/content/drive/MyDrive/Notebooks/Henry/PI/MLOps/datasets/df_movies_clean.csv'
#df_movies = pd.read_csv(filepath)

df_production_companies = normalize_column(df = df_movies,
                                           column = 'production_companies',
                                           key = 'id')


# Define a function to extract the 'id' value from each dictionary in the list
def extract_ids(prodcomps):
  if isinstance(genres, list):
    return [prodcomp['id'] for prodcomp in prodcomps]
  else:
    return []

# Apply the function to the 'genres' column to create a new column with a list of ids
column = 'production_companies'
df_movies[column+'_'+ 'ids'] = df_movies[column].apply(extract_ids)


<ipython-input-133-8c3e262c36a9>:26: FutureWarning: The default value of regex will change from True to False in a future version.
  df[column] = df[column].str.replace("{'name': '", '{"name": "')
<ipython-input-133-8c3e262c36a9>:30: FutureWarning: The default value of regex will change from True to False in a future version.
  df[column] = df[column].str.replace("{'name': ", '{"name": ')


Streaming output truncated to the last 5000 lines.
[{"name": "Studio Dadashow", "id": 32462}]
[]
[{"name": "Stellar Mega Films", "id": 6838}]
[]
[{"name": "Boll Kino Beteiligungs GmbH & Co. KG", "id": 1062}, {"name": "Studio West Productions(VCC)", "id": 23633}, {"name": "Amok II Productions", "id": 34887}]
[{"name": "Event Film Distribution", "id": 22793}]
[{"name": "Kinokompaniya CTB", "id": 886}]
[{"name": "CTB Film Company", "id": 318}]
[{"name": "Jupiter Generale Cinematografica", "id": 1375}, {"name": "Terra-Filmkunst", "id": 4784}, {"name": "Les Films de la Boetie", "id": 18477}]
[]
[{"name": "Jadran Film", "id": 168}, {"name": "Amy International Artists", "id": 84420}, {"name": "National Mutual Australasia Ltd", "id": 84421}]
[{"name": "Magnolia Mae Films", "id": 18188}]
[]
[{"name": "Utopia Pictures", "id": 5332}, {"name": "Beagle II Productions", "id": 11003}]
[{"name": "Drag City", "id": 20107}]
[]
[{"name": "Walt Disney Pictures", "id": 2}]
[{"name": "Walt Disney Pictures",

<ipython-input-133-8c3e262c36a9>:43: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df_aux.append(new_df[i])


In [ ]:
df_movies['production_countries'].iloc[1000]

"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"

In [ ]:
df_movies['spoken_languages'].iloc[1]

"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]"

#### 1.n Store New DFs

In [ ]:
filepath = './datasets/df_genres.csv'
df_genres.to_csv(filepath, index=False)

filepath = './df_production_companies.csv'
df_production_companies.to_csv(filepath, index=False)

filepath = './datasets/df_production_countries.csv'
df_production_countries.to_csv(filepath, index=False)

filepath = './datasets/df_spoken_languagues.csv'
df_spoken_languagues.to_csv(filepath, index=False)

filepath = './datasets/df_movies_clean.csv'
df_movies.to_csv(filepath, index=False)

#### 1.4 Production countries normalization

In [ ]:
df_movies['production_countries'].iloc[0]

"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"

In [ ]:
column = 'production_countries'
#df_movies[column] = df_movies[column].apply(remove_non_ascii)
df_movies[column] = df_movies[column].str.replace('"', "'")
df_movies[column] = df_movies[column].str.replace("{'iso_3166_1': '", '{"iso_3166_1": "')
df_movies[column] = df_movies[column].str.replace("', 'name': '", '", "name": "')
df_movies[column] = df_movies[column].str.replace("', 'name':", '", "name":')
df_movies[column] = df_movies[column].str.replace(", 'name':", ', "name":')
df_movies[column] = df_movies[column].str.replace("{'iso_3166_1': ", '{"iso_3166_1": ')
df_movies[column] = df_movies[column].str.replace("'}", '"}')


# Convert the strings representing dictionaries to actual dictionaries
# Try to parse the JSON string and append the resulting dictionary to the list
df_movies[column] = df_movies[column].apply(lambda x: (print(x),json.loads(x)) if pd.notna(x) else np.nan)

def extract_ids(countries):
  if isinstance(countries, list):
    return [country["iso_3166_1"] for country in countries]
  else:
    return []

column = 'production_countries'
df_movies[column+'_'+ 'ids'] = df_movies[column].apply(extract_ids)


<ipython-input-21-f5c31d7e7801>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df_movies[column] = df_movies[column].str.replace("{'iso_3166_1': '", '{"iso_3166_1": "')
<ipython-input-21-f5c31d7e7801>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_movies[column] = df_movies[column].str.replace("{'iso_3166_1': ", '{"iso_3166_1": ')
<ipython-input-21-f5c31d7e7801>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df_movies[column] = df_movies[column].str.replace("'}", '"}')


Streaming output truncated to the last 5000 lines.
[{"iso_3166_1": "KR", "name": "South Korea"}]
[{"iso_3166_1": "TR", "name": "Turkey"}]
[{"iso_3166_1": "CN", "name": "China"}]
[{"iso_3166_1": "TR", "name": "Turkey"}]
[{"iso_3166_1": "CA", "name": "Canada"}, {"iso_3166_1": "DE", "name": "Germany"}]
[{"iso_3166_1": "US", "name": "United States of America"}, {"iso_3166_1": "CA", "name": "Canada"}]
[{"iso_3166_1": "RU", "name": "Russia"}]
[{"iso_3166_1": "RU", "name": "Russia"}]
[{"iso_3166_1": "FR", "name": "France"}, {"iso_3166_1": "DE", "name": "Germany"}, {"iso_3166_1": "IT", "name": "Italy"}]
[{"iso_3166_1": "DE", "name": "Germany"}]
[{"iso_3166_1": "HR", "name": "Croatia"}, {"iso_3166_1": "GB", "name": "United Kingdom"}]
[{"iso_3166_1": "US", "name": "United States of America"}]
[]
[{"iso_3166_1": "GB", "name": "United Kingdom"}, {"iso_3166_1": "US", "name": "United States of America"}]
[{"iso_3166_1": "US", "name": "United States of America"}]
[{"iso_3166_1": "CL", "name": "Chile"

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,collection_id,release_year,return
0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","(None, [{'iso_3166_1': 'US', 'name': 'United S...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,10194.0,1995,12.451801
1,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","(None, [{'iso_3166_1': 'US', 'name': 'United S...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,NaN,1995,4.043035
2,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.712900,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","(None, [{'iso_3166_1': 'US', 'name': 'United S...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,119050.0,1995,0.000000
3,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"(None, [{'iso_3166_1': 'US', 'name': 'United S...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,NaN,1995,5.090760
4,0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","(None, [{'iso_3166_1': 'US', 'name': 'United S...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,96871.0,1995,0.000000


In [ ]:
# Auxiliar df for processing
aux = df_movies[column].dropna()
aux = aux.apply(pd.Series)
new_df_aux = aux[0]
for i in range(1,aux.shape[1]):
  new_df_aux.append(aux[i])

new_df_aux = new_df_aux.dropna()
#Now that it is all concated in one single serie, lets turn it into a df with 2 features
new_df = new_df_aux.apply(pd.Series)


# Create new df
df_production_countries = new_df.drop_duplicates()

<ipython-input-32-078973dc567f>:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  aux = aux.apply(pd.Series)


,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,"{'iso_3166_1': 'US', 'name': 'United States of...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'iso_3166_1': 'US', 'name': 'United States of...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'iso_3166_1': 'US', 'name': 'United States of...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'iso_3166_1': 'US', 'name': 'United States of...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'iso_3166_1': 'US', 'name': 'United States of...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45371,"{'iso_3166_1': 'CA', 'name': 'Canada'}","{'iso_3166_1': 'DE', 'name': 'Germany'}","{'iso_3166_1': 'GB', 'name': 'United Kingdom'}","{'iso_3166_1': 'US', 'name': 'United States of...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45372,"{'iso_3166_1': 'PH', 'name': 'Philippines'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45373,"{'iso_3166_1': 'US', 'name': 'United States of...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45374,"{'iso_3166_1': 'RU', 'name': 'Russia'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 1.5 Spoken Languages

In [ ]:
df_movies['spoken_languages'].iloc[0]

"[{'iso_639_1': 'en', 'name': 'English'}]"

In [ ]:
column = 'spoken_languages'
#df_movies[column] = df_movies[column].apply(remove_non_ascii)
df_movies[column] = df_movies[column].str.replace('"', "'")
df_movies[column] = df_movies[column].str.replace("{'iso_639_1': '", '{"iso_639_1": "')
df_movies[column] = df_movies[column].str.replace("', 'name': '", '", "name": "')
df_movies[column] = df_movies[column].str.replace("', 'name':", '", "name":')
df_movies[column] = df_movies[column].str.replace(", 'name':", ', "name":')
df_movies[column] = df_movies[column].str.replace("{'iso_639_1': ", '{"iso_639_1": ')
df_movies[column] = df_movies[column].str.replace("'}", '"}')


# Convert the strings representing dictionaries to actual dictionaries
# Try to parse the JSON string and append the resulting dictionary to the list
df_movies[column] = df_movies[column].apply(lambda x: (print(x),parse_json(x)) if pd.notna(x) else np.nan)

def extract_ids(languages):
  if isinstance(languages, list):
    return [language["iso_639_1"] for language in languages]
  else:
    return []

column = 'spoken_languages'
df_movies[column+'_'+ 'ids'] = df_movies[column].apply(extract_ids)


<ipython-input-39-d10b752b998c>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df_movies[column] = df_movies[column].str.replace("{'iso_639_1': '", '{"iso_639_1": "')
<ipython-input-39-d10b752b998c>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_movies[column] = df_movies[column].str.replace("{'iso_639_1': ", '{"iso_639_1": ')
<ipython-input-39-d10b752b998c>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df_movies[column] = df_movies[column].str.replace("'}", '"}')


Streaming output truncated to the last 5000 lines.
[{"iso_639_1": "ko", "name": "한국어/조선말"}]
[{"iso_639_1": "tr", "name": "Türkçe"}]
[{"iso_639_1": "zh", "name": "普通话"}]
[{"iso_639_1": "tr", "name": "Türkçe"}]
[{"iso_639_1": "en", "name": "English"}]
[{"iso_639_1": "en", "name": "English"}]
[{"iso_639_1": "ru", "name": "Pусский"}]
[{"iso_639_1": "ru", "name": "Pусский"}]
[{"iso_639_1": "fr", "name": "Français"}]
[{"iso_639_1": "en", "name": "English"}, {"iso_639_1": "de", "name": "Deutsch"}]
[{"iso_639_1": "en", "name": "English"}, {"iso_639_1": "hr", "name": "Hrvatski"}]
[{"iso_639_1": "en", "name": "English"}]
[{"iso_639_1": "en", "name": "English"}]
[{"iso_639_1": "en", "name": "English"}]
[{"iso_639_1": "en", "name": "English"}]
[{"iso_639_1": "es", "name": "Español"}]
[{"iso_639_1": "en", "name": "English"}]
[{"iso_639_1": "xx", "name": "No Language"}, {"iso_639_1": "en", "name": "English"}]
[{"iso_639_1": "en", "name": "English"}]
[{"iso_639_1": "en", "name": "English"}, {"iso_639

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,collection_id,release_year,return
0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"(None, [{'iso_639_1': 'en', 'name': 'English'}])",Released,NaN,Toy Story,7.7,5415.0,10194.0,1995,12.451801
1,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"(None, [{'iso_639_1': 'en', 'name': 'English'}...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,NaN,1995,4.043035
2,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.712900,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"(None, [{'iso_639_1': 'en', 'name': 'English'}])",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,119050.0,1995,0.000000
3,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"(None, [{'iso_639_1': 'en', 'name': 'English'}])",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,NaN,1995,5.090760
4,0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"(None, [{'iso_639_1': 'en', 'name': 'English'}])",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,96871.0,1995,0.000000


In [ ]:
# Get all unique dictionaries in the column, dropping NaN values
aux = df_movies[column].dropna()
aux = aux.apply(pd.Series)
new_df_aux = aux[0]
for i in range(1,aux.shape[1]):
  new_df_aux.append(aux[i])

new_df_aux = new_df_aux.dropna()
#Now that it is all concated in one single serie, lets turn it into a df with 2 features
new_df = new_df_aux.apply(pd.Series)

# Create the new df
df_spoken_languagues = new_df.drop_duplicates()


<ipython-input-43-4c697d074598>:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  aux = aux.apply(pd.Series)
<ipython-input-43-4c697d074598>:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df_aux.append(aux[i])
<ipython-input-43-4c697d074598>:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df_aux.append(aux[i])
<ipython-input-43-4c697d074598>:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df_aux.append(aux[i])
<ipython-input-43-4c697d074598>:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df_aux.ap

,iso_639_1,name
0,en,English
1,en,English
2,en,English
3,en,English
4,en,English
...,...,...
45370,en,English
45371,en,English
45372,tl,
45373,en,English


## Processing Credits dataset

Similarly, to the approach followed regarding the dictionary columns in movies dataframe, here I choose to create 2 different datasets: casting and crew.

In [3]:
filepath = '/content/drive/MyDrive/Notebooks/Henry/PI/MLOps/datasets/credits.csv'

df_credits = pd.read_csv(filepath)

In [5]:
df_credits['cast'].iloc[0]

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

The Cast column stores all the people working as characters in the movie

In [6]:
df_credits['crew'].iloc[0]

'[{\'credit_id\': \'52fe4284c3a36847f8024f49\', \'department\': \'Directing\', \'gender\': 2, \'id\': 7879, \'job\': \'Director\', \'name\': \'John Lasseter\', \'profile_path\': \'/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f4f\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12891, \'job\': \'Screenplay\', \'name\': \'Joss Whedon\', \'profile_path\': \'/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f55\', \'department\': \'Writing\', \'gender\': 2, \'id\': 7, \'job\': \'Screenplay\', \'name\': \'Andrew Stanton\', \'profile_path\': \'/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f5b\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12892, \'job\': \'Screenplay\', \'name\': \'Joel Cohen\', \'profile_path\': \'/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f61\', \'department\': \'Writing\', \'gender\': 0, \'id\': 12893, \'job\': \'Screenplay\', \'name\': \'A

So, the Crew column stores all the people working on th backstage of the movie, such as director, screenwriter, etc

#### CAST df

A normalized cast df is created following a  iterative process due to the large volume of the data.

Each row of the new df stores the participation of an actor in a given movie. So, movie_id and actor_id create a combined key.

In [4]:
import ast
df_cast = df_credits[['cast','id']]

column = 'cast'
# Convert strings to lists of dictionaries
df_cast['cast'] = df_cast['cast'].apply(ast.literal_eval)

<ipython-input-4-f71a6235c373>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cast['cast'] = df_cast['cast'].apply(ast.literal_eval)


Now, iteratively I will create a normalized cast df. This process takes a lot of time because of the row-to-row process. So it will be perfomer in different partitions. 

In [26]:
df_cast_partition = df_cast.iloc[40001:]
df_cast_partition.head()

,cast,id
40001,"[{'cast_id': 2, 'character': 'Mr Gale', 'credi...",347763
40002,"[{'cast_id': 1001, 'character': 'Jerry', 'cred...",63959
40003,"[{'cast_id': 1, 'character': 'Barry', 'credit_...",21837
40004,"[{'cast_id': 1, 'character': 'Miranda Austin',...",332212
40005,"[{'cast_id': 0, 'character': 'Prashanth', 'cre...",393841


In [27]:
cast_columns = ['movie_id',
                'cast_id',
                'character',
                'credit_id',
                'gender',
                'id',
                'name',
                'order',
                'profile_path']
# Create a new dataframe to store the worker information
new_df = pd.DataFrame(columns=cast_columns)

# Iterate over the rows of the original dataframe
for index, row in df_cast_partition.iterrows():
    movie_id = row['id']
    workers = row['cast']

    # Iterate over the workers list in each row
    for worker in workers:
      cast_id = worker['cast_id']
      character = worker['character']
      credit_id = worker['credit_id']
      gender = worker['gender']
      id = worker['id']
      name = worker['name']
      order = worker['order']
      profile_path =  worker['profile_path']

      # Append a new row to the new dataframe with the movie id, name, and role of the worker
      new_df = new_df.append({'movie_id': movie_id,
                              'cast_id': cast_id,
                              'character': character,
                              'credit_id': credit_id,
                              'gender': gender,
                              'id': id,
                              'name': name,
                              'order': order,
                              'profile_path': profile_path,
                              },
                              ignore_index=True)

# Print the new dataframe
print(new_df)

Streaming output truncated to the last 5000 lines.
<ipython-input-27-ac5e45e5fd01>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'movie_id': movie_id,
<ipython-input-27-ac5e45e5fd01>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'movie_id': movie_id,
<ipython-input-27-ac5e45e5fd01>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'movie_id': movie_id,
<ipython-input-27-ac5e45e5fd01>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'movie_id': movie_id,
<ipython-input-27-ac5e45e5fd01>:30: FutureWarning: The frame.append method is dep

      movie_id cast_id character                 credit_id gender       id  \
0       347763       2   Mr Gale  559b1566c3a36871820004ba      2    73847   
1       347763       3       Fin  559b15839251413ccc0004da      0  1485120   
2       347763       4       Mia  559b15aec3a368719a000512      0  1485121   
3        63959    1001     Jerry  52fe46c3c3a368484e0a2589      0   128182   
4        63959    1002    Grover  52fe46c3c3a368484e0a258d      0   238304   
...        ...     ...       ...                       ...    ...      ...   
52310   227506       2            52fe4ea59251416c7515d7d5      2   544742   
52311   227506       3            52fe4ea59251416c7515d7d9      1  1090923   
52312   227506       4            52fe4ea59251416c7515d7dd      2  1136422   
52313   227506       5            52fe4ea59251416c7515d7e1      0  1261758   
52314   227506       6            52fe4ea59251416c7515d7e5      1    29199   

                     name order                      profile_pa

<ipython-input-27-ac5e45e5fd01>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'movie_id': movie_id,
<ipython-input-27-ac5e45e5fd01>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'movie_id': movie_id,
<ipython-input-27-ac5e45e5fd01>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'movie_id': movie_id,
<ipython-input-27-ac5e45e5fd01>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'movie_id': movie_id,


Storing the partial results (the casting of each partition)

In [28]:
filepath = '/content/drive/MyDrive/Notebooks/Henry/PI/MLOps/datasets/df_cast_5.csv'

new_df.to_csv(filepath, index=False)

In [22]:
new_df.head()

,movie_id,cast_id,character,credit_id,gender,id,name,order,profile_path
0,76800,3,Joe May,52fe494ec3a368484e124b9f,2,1117,Dennis Farina,0,/o649nAtUimSZxEV6xa5JbMSgsED.jpg
1,76800,4,Jenny Rapp,52fe494ec3a368484e124ba3,1,77368,Jamie Anne Allman,1,/9NlzBDl1xTVmZ5hPlOkhPG5r66h.jpg
2,76800,5,Stanley Buczkowski,52fe494ec3a368484e124ba7,2,181514,Ian Barford,2,/p41HOFgCapwGygoks1KYitA04D3.jpg
3,76800,6,Angelina Rapp,52fe494ec3a368484e124bab,1,135164,Meredith Droeger,3,/euwiv6HMsc7WO8JayoKxV3HZKXv.jpg
4,76800,7,Billy,52fe494ec3a368484e124baf,2,10486,Chelcie Ross,4,/gtUOc36bK5MrjdRPQJi1b0cc0LT.jpg


#### CREW df

Similarly, a normalized crew df is created, following the same iterative process because the large volume of the data.

Each row of the new df stores the participation of a worker in a given movie. So, movie_id and crew_id create a combined key.


In [19]:
import ast
filepath = './datasets/credits.csv'

df_credits = pd.read_csv(filepath)
df_crew = df_credits[['crew','id']]


def replace_none_with_null(obj):
    """
    Recursively replaces all occurrences of the string 'None' with the value None.
    """
    if isinstance(obj, dict):
        return {k: replace_none_with_null(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [replace_none_with_null(elem) for elem in obj]
    elif isinstance(obj, str) and obj.lower() == 'none':
        return None
    else:
        return obj



column = 'crew'
# Convert strings to lists of dictionaries
df_crew[column] = df_crew[column].apply(lambda x: replace_none_with_null(ast.literal_eval(x)))


<ipython-input-19-e416ff8bc7af>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_crew[column] = df_crew[column].apply(lambda x: replace_none_with_null(ast.literal_eval(x)))


,crew,id
0,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [36]:
df_crew_partition = df_crew.iloc[30001:]
df_crew_partition.head()

,crew,id
30001,"[{'credit_id': '52fe45279251416c7504d8b3', 'de...",12826
30002,"[{'credit_id': '52fe49039251416c9109fc9d', 'de...",84628
30003,"[{'credit_id': '544a9b94c3a3680fbb004006', 'de...",191322
30004,"[{'credit_id': '52fe4665c3a368484e08e479', 'de...",61803
30005,"[{'credit_id': '52fe4898c3a368484e1012a5', 'de...",73369


In [37]:
crew_columns = ['movie_id',
                'credit_id',
                'department',
                'gender',
                'id',
                'job',
                'name',
                'profile_path']

# Create a new dataframe to store the worker information
new_df = pd.DataFrame(columns=crew_columns)

# Iterate over the rows of the original dataframe
for index, row in df_crew_partition.iterrows():
    movie_id = row['id']
    workers = row['crew']

    # Iterate over the workers list in each row
    for worker in workers:
      credit_id = worker['credit_id']
      department = worker['department']
      gender = worker['gender']
      id = worker['id']
      job = worker['job']
      name = worker['name']
      profile_path =  worker['profile_path']

      # Append a new row to the new dataframe with the movie id, name, and role of the worker
      new_df = new_df.append({'movie_id': movie_id,
                              'credit_id': credit_id,
                              'department' : department,
                              'gender': gender,
                              'id': id,
                              'job':job,
                              'name': name,
                              'profile_path': profile_path,
                              },
                              ignore_index=True)

# Print the new dataframe
print(new_df)

Streaming output truncated to the last 5000 lines.
<ipython-input-37-a40abc194728>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'movie_id': movie_id,
<ipython-input-37-a40abc194728>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'movie_id': movie_id,
<ipython-input-37-a40abc194728>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'movie_id': movie_id,
<ipython-input-37-a40abc194728>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'movie_id': movie_id,
<ipython-input-37-a40abc194728>:29: FutureWarning: The frame.append method is dep

       movie_id                 credit_id  department gender       id  \
0         12826  52fe45279251416c7504d8b3   Directing      2    15196   
1         12826  52fe45279251416c7504d8b9     Writing      0  1031924   
2         12826  52fe45279251416c7504d8bf     Writing      0  1031925   
3         12826  52fe45279251416c7504d8c5     Writing      0    25316   
4         12826  52fe45279251416c7504d8cb     Writing      0    36874   
...         ...                       ...         ...    ...      ...   
114214    67758  52fe4776c3a368484e0c8399       Sound      0   549356   
114215    67758  52fe4776c3a368484e0c839f      Camera      2    58818   
114216   227506  533bccebc3a36844cf0011a7   Directing      0  1085341   
114217   227506  58ebbc26925141281908aa0a  Production      2  1195656   
114218   461257  593e676c92514105b702e68e   Directing      0  1344495   

                            job                 name  \
0                      Director       David Hemmings   
1          

<ipython-input-37-a40abc194728>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'movie_id': movie_id,
<ipython-input-37-a40abc194728>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'movie_id': movie_id,


In [38]:
# Store partial result
filepath = './datasets/df_crew_4.csv'
new_df.to_csv(filepath, index=False)

### Merging Cast and Crew partitions

Hereafter, the different partitions of crew_df and cast_df are combined to get two unique df.

#### Crew DF

In [6]:
filepath = './datasets/'
df1 = pd.read_csv(filepath+'df_crew_1.csv')
df2 = pd.read_csv(filepath+'df_crew_2.csv')
df3 = pd.read_csv(filepath+'df_crew_3.csv')
df4 = pd.read_csv(filepath+'df_crew_4.csv')
# Merge the 5 DataFrames on a common column
merged_df = df1.append(df2)\
               .append(df3)\
               .append(df4)

# Display the merged DataFrame
merged_df.head()

<ipython-input-6-e0e405a3fe66>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = df1.append(df2)\
<ipython-input-6-e0e405a3fe66>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  .append(df3)\
<ipython-input-6-e0e405a3fe66>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  .append(df4)


,movie_id,credit_id,department,gender,id,job,name,profile_path
0,862,52fe4284c3a36847f8024f49,Directing,2,7879,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,862,52fe4284c3a36847f8024f4f,Writing,2,12891,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
2,862,52fe4284c3a36847f8024f55,Writing,2,7,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
3,862,52fe4284c3a36847f8024f5b,Writing,2,12892,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
4,862,52fe4284c3a36847f8024f61,Writing,0,12893,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg


In [7]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 464299 entries, 0 to 114218
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   movie_id      464299 non-null  int64 
 1   credit_id     464299 non-null  object
 2   department    464299 non-null  object
 3   gender        464299 non-null  int64 
 4   id            464299 non-null  int64 
 5   job           464299 non-null  object
 6   name          464299 non-null  object
 7   profile_path  95097 non-null   object
dtypes: int64(3), object(5)
memory usage: 31.9+ MB


In [8]:
merged_df.to_csv(filepath+'df_crew.csv', index=False)

#### CAST DF

In [10]:
df1 = pd.read_csv(filepath+'df_cast_1.csv')
df2 = pd.read_csv(filepath+'df_cast_2.csv')
df3 = pd.read_csv(filepath+'df_cast_3.csv')
df4 = pd.read_csv(filepath+'df_cast_4.csv')
df5 = pd.read_csv(filepath+'df_cast_5.csv')
# Merge the 5 DataFrames on a common column
merged_df = df1.append(df2)\
               .append(df3)\
               .append(df4)\
               .append(df5)

# Display the merged DataFrame
merged_df.head()

<ipython-input-10-4ad862f5422e>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = df1.append(df2)\
<ipython-input-10-4ad862f5422e>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  .append(df3)\
<ipython-input-10-4ad862f5422e>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  .append(df4)\
<ipython-input-10-4ad862f5422e>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  .append(df5)


,movie_id,cast_id,character,credit_id,gender,id,name,order,profile_path
0,862,14,Woody (voice),52fe4284c3a36847f8024f95,2,31,Tom Hanks,0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,862,15,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2,12898,Tim Allen,1,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
2,862,16,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2,7167,Don Rickles,2,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
3,862,17,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2,12899,Jim Varney,3,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
4,862,18,Rex (voice),52fe4284c3a36847f8024fa5,2,12900,Wallace Shawn,4,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg


In [11]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 562385 entries, 0 to 52314
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   movie_id      562385 non-null  int64 
 1   cast_id       562385 non-null  int64 
 2   character     536603 non-null  object
 3   credit_id     562385 non-null  object
 4   gender        562385 non-null  int64 
 5   id            562385 non-null  int64 
 6   name          562385 non-null  object
 7   order         562385 non-null  int64 
 8   profile_path  388575 non-null  object
dtypes: int64(5), object(4)
memory usage: 42.9+ MB


In [12]:
merged_df.to_csv(filepath+'df_cast.csv', index=False)